In [2]:
import pandas as pd
import tqdm

In [4]:
brute_sentences_df = pd.read_csv('./data/dataset/brute_sentences.csv')
train_station_df = pd.read_csv('./data/liste-des-gares.csv', sep=';')

In [23]:
train_station_df.loc[train_station_df.index == 2]

,CODE_UIC,LIBELLE,FRET,VOYAGEURS,CODE_LIGNE,RG_TRONCON,PK,COMMUNE,DEPARTEMEN,IDRESEAU,IDGAIA,X_L93,Y_L93,X_WGS84,Y_WGS84,C_GEO,Geo Point,Geo Shape
2,87382218,La Défense,N,O,973000,1,008+295,PUTEAUX,HAUTS-DE-SEINE,4648,c0d4c69a-f312-11e3-90ff-015864e0362d,644164.1873,6.866206e+06,2.238472,48.893437,"48.89343723770491, 2.2384716845345993","48.89343723768303, 2.2384716845345984","{""coordinates"": [2.238471684534598, 48.8934372..."


In [26]:
clean_df = []

for index, train_station in enumerate(tqdm.tqdm(train_station_df["LIBELLE"], unit="train station")):
    departure = train_station

    for sentence in brute_sentences_df["sentence"]:
        arrival = train_station_df.sample()
        while (arrival["LIBELLE"].values[0] == departure):
            arrival = train_station_df.sample()
        
        new_sentence = sentence.replace('[depart]', departure)
        new_sentence = new_sentence.replace('[arrival]', arrival["LIBELLE"].values[0])

        new_row = {
            'sentence': new_sentence,
            'departure_train_station_id': index,
            'departure': departure,
            'arrival': arrival["LIBELLE"].values[0],
            'arrival_train_station_id': arrival.index[0],
        }
    
        clean_df.append(new_row)

clean_df = pd.DataFrame(clean_df)

  0%|                                                          | 0/3884 [00:00<?, ?train station/s]

100%|███████████████████████████████████████████████| 3884/3884 [01:10<00:00, 55.17train station/s]


In [35]:
clean_df.loc[
    (clean_df["arrival_train_station_id"] == 1) & 
    (clean_df["departure_train_station_id"] == 2)
]

,sentence,departure_train_station_id,departure,arrival,arrival_train_station_id


In [5]:
clean_df.to_csv('./data/dataset/clean_sentences.csv', index=False)

In [6]:
clean_df_shuffled = clean_df.sample(frac=1)

In [7]:
train_df = clean_df_shuffled[:int(len(clean_df_shuffled)*0.7)]
valid_df = clean_df_shuffled[int(len(clean_df_shuffled)*0.7):int(len(clean_df_shuffled)*0.85)]
test_df = clean_df_shuffled[int(len(clean_df_shuffled)*0.85):]

In [8]:
train_df.to_csv('./data/dataset/train.csv', index=False)
valid_df.to_csv('./data/dataset/valid.csv', index=False)
test_df.to_csv('./data/dataset/test.csv', index=False)